# Project 5 - Regularisation and Pareto fronts

An important aspect in the design of Stellarators is the complexity of the coils. The simpler the coil shapes, the cheaper and easier they are to build. Two measures of complexity are given by the curvature and the torsion of the coils. The vacuum optimisation code allows the user to add penalty terms to the objective that penalise large curvature or torsion.

Let's begin by setting up the objective.

In [ ]:
from pyplasmaopt import *
nfp = 2
(coils, currents, expansion_axis, eta_bar) = get_24_coil_data(Nt_coils=5, Nt_ma=5, nfp=nfp, ppp=10, at_optimum=False)
stellarator = CoilCollection(coils, currents, nfp, True)
iota_target = 0.103
coil_length_target = 4.398229715025710
magnetic_axis_length_target = 6.356206812106860
eta_bar = -2.25

obj = SimpleNearAxisQuasiSymmetryObjective(
        stellarator, expansion_axis, iota_target, eta_bar=eta_bar,
        coil_length_target=coil_length_target, magnetic_axis_length_target=magnetic_axis_length_target)

Let's begin by running the optimisation without any regularisation.

In [ ]:
from scipy.optimize import minimize

def scipy_fun(x):
    obj.update(x)
    res = obj.res
    dres = obj.dres
    return res, dres

obj.clear_history()
res = minimize(scipy_fun, obj.x0, jac=True, method='bfgs', tol=1e-20,
               options={"maxiter": 1000},
               callback=obj.callback)

Let's have a look at the resulting Stellarator

In [ ]:
plot_stellarator(stellarator, axis=expansion_axis)

We can see that some of the coils have segments of high curvature.

## Tasks

1) Let's quantify the simplicity of the coils. For a curve $\Gamma: [0, 1) \to \mathbb{R}, \phi\mapsto \Gamma(\phi)$, look up the formula for the curvature of this curve. Then complete the function below to compute the curvature of a coil

    def kappa(coil):
        # array with shape (N, 3) giving the (x, y, z) coordinates of N points along the curve
        gamma = coil.gamma
        # array with shape (N, 3) giving the derivative of the curve 
        dgamma_by_dphi = = coil.dgamma_by_dphi[:, 0, :]at those same N points
        # array with shape (N, 3) giving the second derivative of the curve at those same N points
        d2gamma_by_dphidphi = = coil.dgamma_by_dphi[:, 0, 0, :]
        
        kappa = ???
        
        return kappa
   
You can check your code by computing the curvature of the initial coils. They should all have curvature $\approx 1.42857143$. Now use this function to compute the maximum curvature on all 6 coils obtained from the unregularised optimisation.

2) By setting `obj.curvature_weight = 1e-3` we can add a penalty on the curvature. Using the previous optimisation result as initial guess, what do the resulting coils look like, how small is the objective, and what is the maximum curvature of the coils?

        

2) Clearly there is a trade-off between simplicity of the coils and achieving quasi symmetry on axis. Run the optimisation for a range of curvature penalties. For each run, take note of the maximum curvature in the coils and the quasi-symmetry and iota misfit. The latter can be obtained via

    obj.res1 + obj.res4
    


## Solutions

### Task 1

In [ ]:
def kappa(coil):
    # array with shape (N, 3) giving the (x, y, z) coordinates of N points along the curve
    gamma = coil.gamma
    # array with shape (N, 3) giving the derivative of the curve at those same N points
    dgamma_by_dphi = coil.dgamma_by_dphi[:, 0, :]
    # array with shape (N, 3) giving the second derivative of the curve at those same N points
    d2gamma_by_dphidphi = coil.d2gamma_by_dphidphi[:, 0, 0, :]

    kappa = (np.linalg.norm(np.cross(dgamma_by_dphi, d2gamma_by_dphidphi), axis=1)/np.linalg.norm(dgamma_by_dphi, axis=1)**3)

    return kappa

In [ ]:
obj.update(obj.x0)
print('Maximum curvature for each initial coil:\n', [np.max(kappa(c)) for c in coils])
obj.update(res.x)
print('Maximum curvature for each optimised coil (without regularisation):\n', [np.max(kappa(c)) for c in coils])

### Task 2

In [ ]:
obj.curvature_weight = 1e-3
obj.clear_history()
res_reg = minimize(scipy_fun, res.x, jac=True, method='bfgs', tol=1e-20,
                   options={"maxiter": 500},
                   callback=obj.callback)

In [ ]:
print('Maximum curvature for each optimised coil (with regularisation):\n', [np.max(kappa(c)) for c in coils])

### Task 3

In [ ]:
qsmisfits = []
maxcurvatures = []
ws = [1e-9, 1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3]
xmins = []

for w in ws:
    obj.curvature_weight = w
    obj.clear_history()
    res_reg = minimize(scipy_fun, res.x, jac=True, method='bfgs', tol=1e-20,
                       options={"maxiter": 1000, "maxcor": 100},
                       callback=obj.callback)
    qsmisfits.append(obj.res1 + obj.res4)
    maxcurvatures.append(max(np.max(c.kappa) for c in coils))
    xmins.append(res_reg.x.copy())

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.semilogx(qsmisfits, maxcurvatures)
plt.show()